### 1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

### 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

### 3번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 64, out_channel = 128, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2, padding=1))

### 4번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 4 × 4 × 128 → batch_size × 2048
전결합층(뉴런 625개) + 활성화 함수 ReLU

### 5번 레이어 : 전결합층(Fully-Connected layer)
전결합층(뉴런 10개) + 활성화 함수 Softmax

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

learning_rage = 0.001
training_epochs = 15
batch_size = 100

#dataloader
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                          train = False,
                          transform = transforms.ToTensor(),
                          download=True)

data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size =batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [3]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.keep_prob = 0.5
        #L1 image in shape = (?,28,28,1)
        #    conv -> (?, 28, 28, 32)
        #    pool -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2)
        )

        #L2 image in shape = (?, 14,14,32)
        #   conv -> (?,14,14,64)
        #   pool -> (?,7,7,64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3 ,stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2)
        )

        #L3 image in shape = (?, 7,7,64)
        #   conv -> (?, 7,7,128)
        #   pool -> (?, 4,4,128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding =1)
        )

        #Fully connected Layer 4*4*128
        self.fc1 = torch.nn.Linear(4*4*128, 625, bias=True)
        torch.nn.init.xavier_uniform(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1-self.keep_prob)
        )

        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform(self.fc2.weight)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [4]:
#
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rage)

total_batch = len(data_loader)
print("total batch  ", total_batch)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)

        optimizer.zero_grad()
        hypothesys = model(X)
        cost = criterion(hypothesys, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print("epoch {:>4} cost={:>.9f}".format(epoch+1, avg_cost))




C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_20768/246938280.py:34: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc1.weight)
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_20768/246938280.py:42: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc2.weight)


total batch   600
epoch    1 cost=0.199988663
epoch    2 cost=0.053253159
epoch    3 cost=0.038322367
epoch    4 cost=0.029754195
epoch    5 cost=0.023712467
epoch    6 cost=0.020826468
epoch    7 cost=0.016701790
epoch    8 cost=0.015272898
epoch    9 cost=0.012853594
epoch   10 cost=0.011772370
epoch   11 cost=0.009190964
epoch   12 cost=0.010460556
epoch   13 cost=0.008212022
epoch   14 cost=0.008508514
epoch   15 cost=0.006829794


In [5]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, dim = 1) ==Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy : ", accuracy.item())


C:\anaconda3\envs\pytorch04\lib\site-packages\torchvision\datasets\mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\anaconda3\envs\pytorch04\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy :  0.9801999926567078
